# unfinished

# Import

In [1]:
!pip3 install textblob

In [2]:
import re
import os
import numpy as np
import pandas as pd
from string import punctuation
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize
from bs4 import BeautifulSoup
from nltk.stem.snowball import EnglishStemmer

In [3]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Gebruiker\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Gebruiker\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Gebruiker\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
from textblob import Word

In [5]:
df = pd.read_csv ('IMDB-Dataset.csv')

In [6]:
df.head(10)

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
5,"Probably my all-time favorite movie, a story o...",positive
6,I sure would like to see a resurrection of a u...,positive
7,"This show was an amazing, fresh & innovative i...",negative
8,Encouraged by the positive comments about this...,negative
9,If you like original gut wrenching laughter yo...,positive


# Set Clean-up

In [7]:
def remove_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

#Removing the square brackets
def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)

def remove_special_characters(text, remove_digits=True):
    pattern=r'[^a-zA-z0-9\s]'
    text=re.sub(pattern,'',text)
    return text

#Removing the noisy text
def clean_text(text):
    text = remove_html(text)
    text = remove_between_square_brackets(text)
    text = remove_special_characters(text)
    return text

def transformations(df):
    #Punctuation removal
    df['review'] = df['review'].apply(lambda words: [x for x in words if not x in punctuation])
    #Number removal
    df['review'] = df['review'].apply(lambda words: [x for x in words if not x.isdigit()])
    #Stopword removal
    df['review'] = df['review'].apply(lambda words: [x for x in words if x not in stopwords.words('english')])
    #Frequent word removal
    temp = df['review'].apply(lambda words: " ".join(words))
    freq = pd.Series(temp).value_counts()[:10]
    df['review'] = df['review'].apply(lambda words: [x for x in words if x not in freq.keys()])
    #Lemmatization
    df['review'] = df['review'].apply(lambda words: " ".join([Word(x).lemmatize() for x in words]))
    return df

#Apply function on review column
df['review'] = df['review'].apply(clean_text)



In [8]:
df.dropna()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production The filming tech...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically theres a family where a little boy J...,negative
4,Petter Matteis Love in the Time of Money is a ...,positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,Bad plot bad dialogue bad acting idiotic direc...,negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,Im going to have to disagree with the previous...,negative


# Split set

In [9]:
df.head(10)

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production The filming tech...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically theres a family where a little boy J...,negative
4,Petter Matteis Love in the Time of Money is a ...,positive
5,Probably my alltime favorite movie a story of ...,positive
6,I sure would like to see a resurrection of a u...,positive
7,This show was an amazing fresh innovative ide...,negative
8,Encouraged by the positive comments about this...,negative
9,If you like original gut wrenching laughter yo...,positive


In [10]:
#positive = df[df['sentiment'] == 'positive']

In [11]:
#negative = df[df['sentiment'] == 'negative']

In [12]:
train, valid, test = np.split(df.sample(frac=1, random_state=42), [int(.6*len(df)), int(.8*len(df))])

In [13]:
###

In [14]:
X_train = train.review
Y_train = train.sentiment

In [15]:
X_valid = valid.review
Y_valid = valid.sentiment

In [16]:
X_test = test.review
Y_test = test.sentiment

In [17]:
X_train.head(30)

33553    I really liked this Summerslam due to the look...
9427     Not many television shows appeal to quite as m...
199      The film quickly gets to a major chase scene w...
12447    Jane Austen would definitely approve of this o...
39489    Expectations were somewhat high for me when I ...
42724    Ive watched this movie on a fairly regular bas...
10822    For once a story of hope highlighted over the ...
49498    Okay I didnt get the Purgatory thing the first...
4144     I was very disappointed with this series It ha...
36958    The first 30 minutes of Tinseltown had my fing...
43106    jeez this was immensely boring the leading man...
38695    Great just great The West Coast got Dirty Harr...
6188     Its made in 2007 and the CG is bad for a movie...
1414     This movie stinks majorly The only reason I ga...
18471    We can start with the wooden acting but this f...
29282    This movie starts off somewhat slowly and gets...
15177    This is a slightly uneven entry with one stand.

In [18]:
clf = Pipeline(steps =[
('preprocessing', CountVectorizer()),
('classifier', LogisticRegression(dual=False,max_iter=2000))
])

clf.fit(X_train, Y_train)

Pipeline(steps=[('preprocessing', CountVectorizer()),
                ('classifier', LogisticRegression(max_iter=2000))])

In [19]:
clf.score(X_valid, Y_valid)
clf.score(X_test,Y_test)

0.8794

In [23]:
p = clf.predict(X_test)

array(['negative', 'positive', 'positive', ..., 'negative', 'positive',
       'positive'], dtype=object)

In [25]:
from collections import Counter
Counter(p)

Counter({'negative': 4954, 'positive': 5046})